In [1]:
import plotly

In [17]:
import plotly.graph_objects as go

labels = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
values = [4500, 2500, 1053, 500]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=0.5)])

fig2 = go.Figure(go.Bar(x=values, y=labels, name="Vari gas", orientation='h'))

fig.show()
fig2.show()

In [26]:
from plotly.subplots import make_subplots


fig = make_subplots(rows=1,cols=2)
fig.add_trace(
    go.Scatter(x=[1,2,3], y=[4,5,6]),row=1,col=1)
fig.add_trace(
    go.Scatter(x=[20,30,40], y=[50,60,70]),row=1,col=2)

fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")

In [130]:
import pandas as pd
istanti = ['2021-08-28 17:50:50','2021-08-28 17:51:10','2021-08-28 17:51:30','2021-08-28 17:51:50','2021-08-28 17:52:20','2021-08-28 17:52:50','2021-08-28 17:53:10','2021-08-28 17:54:10','2021-08-28 17:54:50','2021-08-28 17:55:10']

positives = [6,4,4,3,2,1,0,3,5,3]
negatives = [2,2,1,3,6,1,6,4,5,1]
text = ["Bello de","Super!","Va bene","Continua così","Mitico!","Male","Malino","Non bene","Ahia ahiai..","Non bene"]
polarity = [1,1,1,1,1,-1,-1,-1,-1,-1]
dataset = {'polarity':polarity} #data è stato tolto
df = pd.DataFrame(dataset, index = istanti)
print(df) 

                     polarity
2021-08-28 17:50:50         1
2021-08-28 17:51:10         1
2021-08-28 17:51:30         1
2021-08-28 17:51:50         1
2021-08-28 17:52:20         1
2021-08-28 17:52:50        -1
2021-08-28 17:53:10        -1
2021-08-28 17:54:10        -1
2021-08-28 17:54:50        -1
2021-08-28 17:55:10        -1


In [124]:
df['polarity'].value_counts()

 1    5
-1    5
Name: polarity, dtype: int64

In [148]:
df.index = pd.to_datetime(df.index)
#df = df.set_index(['data'])
#del df['data']
#print(df)
result = df.groupby([pd.Grouper(freq='5min'),'polarity'])
print(result.count())
#result.agg({'Conteggio':'count'})
result = result.agg({'polarity':'count'}).rename(columns={'polarity':'polarity Count'})
print(result)

Empty DataFrame
Columns: []
Index: [(2021-08-28 17:50:00, -1), (2021-08-28 17:50:00, 1), (2021-08-28 17:55:00, -1)]
                              polarity Count
                    polarity                
2021-08-28 17:50:00 -1                     4
                     1                     5
2021-08-28 17:55:00 -1                     1


In [161]:
#ORO, adesso sto capendo perché vengono fatti un po' di
#magheggi con reset_index(),stack(),unstack(fill_value=0)
print(result)
result2 = result.reset_index()
print(result2)
print(result.unstack(fill_value=0).stack().reset_index())

print(result.unstack(fill_value=0).stack().reset_index().rename(columns={'level_0':'istante'}))
result_final = result.unstack(fill_value=0).stack().reset_index().rename(columns={'level_0':'istante'})


                              polarity Count
                    polarity                
2021-08-28 17:50:00 -1                     4
                     1                     5
2021-08-28 17:55:00 -1                     1
              level_0  polarity  polarity Count
0 2021-08-28 17:50:00        -1               4
1 2021-08-28 17:50:00         1               5
2 2021-08-28 17:55:00        -1               1
              level_0  polarity  polarity Count
0 2021-08-28 17:50:00        -1               4
1 2021-08-28 17:50:00         1               5
2 2021-08-28 17:55:00        -1               1
3 2021-08-28 17:55:00         1               0
              istante  polarity  polarity Count
0 2021-08-28 17:50:00        -1               4
1 2021-08-28 17:50:00         1               5
2 2021-08-28 17:55:00        -1               1
3 2021-08-28 17:55:00         1               0


In [169]:
#sintassi interessante: (non ancora compresa al 100%)
dominioTempo = result_final["istante"][result_final['polarity']==1].reset_index(drop=True)

print(dominioTempo)

0   2021-08-28 17:50:00
1   2021-08-28 17:55:00
Name: istante, dtype: datetime64[ns]


In [191]:
#Ora che ho i dati potrei procedere a mostrare i grafici!!! -> BOMBA
fig = make_subplots(rows=2, cols=2, specs=[[{"type":"scatter"},{"type":"pie"}],
                                           [{},{}]]) #(1)

fig.add_trace(go.Scatter(
    x=dominioTempo,
    y=result_final['polarity Count'][result_final['polarity']==-1].reset_index(drop=True),
    name="Negative",
    line_color="#00897b",
    opacity=0.8),
    row=1,col=1
    )
fig.add_trace(go.Scatter(
    x=dominioTempo,
    y=result_final['polarity Count'][result_final['polarity']==1].reset_index(drop=True),
    name="Positive",
    line_color="#039be5",
    opacity=0.8),
    row=1,col=1
    )
legenda = ['Positive','Negative','Neutral']
fig.add_trace(go.Pie(labels=legenda,values=[5,5,10],
                     hole=0.3),row=1,col=2)



#(1) Per alcuni tipi di grafici è obbligatorio specificare il tipo.
#Nella riga 1 colonna 2 ho dovuto specificare il tipo 'pie' perchè 
#altrimenti non sarebbe stato visualizzato.
#Notare che nella riga 2 non ho specificato tipi di grafici.

